In [10]:
from abc import ABC, abstractmethod
import hashlib
from ecdsa import SigningKey, VerifyingKey, SECP256k1, BadSignatureError
from ecdsa.util import sigencode_string_canonize
from bech32 import bech32_encode, convertbits
import base64
from eth_keys import keys as eth_keys
from dataclasses import dataclass
import json

### Verifier Interface

In [11]:
class SignatureVerifier(ABC):
    """
    Abstract class to define signature verification and address construction
    from different chains types
    """
    @abstractmethod
    def verify_signature(self, public_key: str, message: str, signature: str):
        pass

    @abstractmethod
    def generate_address(self, public_key: str):
        pass

### Signers

In [12]:
def sign_cosmos(private_key: str, message: str) -> str:
    """
    Signs a message from a cosmos account
    """
    pk = SigningKey.from_string(bytes.fromhex(private_key), curve=SECP256k1, hashfunc=hashlib.sha256)
    return pk.sign(message.encode()).hex()

def sign_ethereum(private_key: str, message: str) -> str:
    """
    Signs a message from an etherum account
    """
    pk = eth_keys.PrivateKey(bytes.fromhex(private_key))
    return str(pk.sign_msg(message.encode()))

### Verifiers

In [13]:
@dataclass
class CosmosVerifier(SignatureVerifier):
    """
    Verifier for cosmos addresses (coin type 118)
    """
    bech_prefix: str

    def verify_signature(self, pubkey_b64: str, message_amino: str, sig_b64: str):
        """
        Verifies a signature from a cosmos account
        """
        pubkey_bz = base64.b64decode(pubkey_b64)
        verifying_key = VerifyingKey.from_string(pubkey_bz, curve=SECP256k1, hashfunc=hashlib.sha256)
        sig_bz = base64.b64decode(sig_b64)
        try:
            return verifying_key.verify(sig_bz, message_amino.encode())
        except BadSignatureError:
            return False

    def generate_address(self, pubkey_b64: str):
        """
        Generates a cosmos address from a public key
        """
        pubkey_bz = base64.b64decode(pubkey_b64)
        s = hashlib.new("sha256", pubkey_bz).digest()
        r = hashlib.new("ripemd160", s).digest()
        r = convertbits(r, 8, 5)
        if not r:
            raise ValueError("unable to generate address")
        return bech32_encode(self.bech_prefix, r)
    
class EthereumVerifier(SignatureVerifier):
    """
    Verifier for ethereum addresses (coin type 60)
    """

    def verify_signature(self, public_key: str, message: str, signature: str):
        """
        Verifies a signature from an ethereum account
        """
        verifying_key = eth_keys.PublicKey(bytes.fromhex(public_key))
        signature_obj = eth_keys.Signature(bytes.fromhex(signature))
        try:
            verified = verifying_key.verify_msg(message.encode(), signature_obj)
            return verified
        except Exception:
            return False

    def generate_address(self, public_key: str):
        """
        Generates an ethereum address from a public key
        """
        return eth_keys.PublicKey(bytes.fromhex(public_key)).to_address()


### Example

In [23]:
import json

# The idea is that the user will sign their stride_address on the frontend, and call a backend function `link(pubkey, prefix, signature, stride_address)`
# On the backend, we will rebuild `signed_doc` using only those 4 params, and try to verify `signature(recreated_signed_doc)`` against `pubkey`.
# Once that's successful, we will derive the signer's address using `prefix` and `pubkey`.
# That will give us all the confirmation we need to link the 2 addresses.

signed_doc = {
    "value": {
        "signed": {
            "chain_id": "",
            "account_number": "0",
            "sequence": "0",
            "fee": {"gas": "0", "amount": []},
            "msgs": [
                {
                    "type": "sign/MsgSignData",
                    "value": {
                        "signer": "cosmos1uspq8jesuu0f6uh6dez2lktnhuxrplzasuywag",
                        "data": "c3RyaWRlMXVzcHE4amVzdXUwZjZ1aDZkZXoybGt0bmh1eHJwbHphbmh5amZ5",
                    },
                }
            ],
            "memo": "",
        },
        "signature": {
            "pub_key": {"type": "tendermint/PubKeySecp256k1", "value": "A44GECf+ZykwzE9LxNBOTH+0pv3jRbl2Oz86vj7WeLUm"},
            "signature": "bT4/+FjxW435j9eNIJG+mJNF01/e0R3guqXKqHm+emQOEggBj7ygRO2c/iHbXuNilekG4HAlblfL+pATKdgHjQ==",
        },
    }
}

signer = "cosmos1uspq8jesuu0f6uh6dez2lktnhuxrplzasuywag"
stride_address = "stride1uspq8jesuu0f6uh6dez2lktnhuxrplzanhyjfy"

message = signed_doc["value"]["signed"]
message_amino = json.dumps(message, separators=(",", ":"), sort_keys=True)

pubkey_b64 = signed_doc["value"]["signature"]["pub_key"]["value"]
sig_b64 = signed_doc["value"]["signature"]["signature"]

In [24]:
cosmos_client = CosmosVerifier(bech_prefix="cosmos")

verification = cosmos_client.verify_signature(pubkey_b64, message_amino, sig_b64)
address_from_pubkey = cosmos_client.generate_address(pubkey_b64)

print("Signature:", sig_b64)
print("Verification:", verification)
print("Address Generated:", address_from_pubkey)
print("Address Matched:", address_from_pubkey == signer)

Signature: bT4/+FjxW435j9eNIJG+mJNF01/e0R3guqXKqHm+emQOEggBj7ygRO2c/iHbXuNilekG4HAlblfL+pATKdgHjQ==
Verification: True
Address Generated: cosmos1uspq8jesuu0f6uh6dez2lktnhuxrplzasuywag
Address Matched: True


In [17]:
cosmos_client = CosmosVerifier(bech_prefix="cosmos")

verification = cosmos_client.verify_signature(pubkey_b64, message_amino, sig_b64)
address_from_pubkey = cosmos_client.generate_address(pubkey_b64)

print("Signature:", sig_b64)
print("Verification:", verification)
print("Address Generated:", address_from_pubkey)
print("Address Matched:", address_from_pubkey == cosmos_address)

Signature: bT4/+FjxW435j9eNIJG+mJNF01/e0R3guqXKqHm+emQOEggBj7ygRO2c/iHbXuNilekG4HAlblfL+pATKdgHjQ==
Verification: True
Address Generated: cosmos1uspq8jesuu0f6uh6dez2lktnhuxrplzasuywag
Address Matched: False


In [18]:
cosmos_client = CosmosVerifier(bech_prefix="cosmos")

verification = cosmos_client.verify_signature(pubkey_b64, message_amino, sig_b64)
address_from_pubkey = cosmos_client.generate_address(pubkey_b64)

print("Signature:", sig_b64)
print("Verification:", verification)
print("Address Generated:", address_from_pubkey)
print("Address Matched:", address_from_pubkey == cosmos_address)

Signature: bT4/+FjxW435j9eNIJG+mJNF01/e0R3guqXKqHm+emQOEggBj7ygRO2c/iHbXuNilekG4HAlblfL+pATKdgHjQ==
Verification: True
Address Generated: cosmos1uspq8jesuu0f6uh6dez2lktnhuxrplzasuywag
Address Matched: False


In [19]:
cosmos_client = CosmosVerifier(bech_prefix="cosmos")

verification = cosmos_client.verify_signature(pubkey_b64, message_amino, sig_b64)
address_from_pubkey = cosmos_client.generate_address(pubkey_b64)

print("Signature:", sig_b64)
print("Verification:", verification)
print("Address Generated:", address_from_pubkey)
print("Address Matched:", address_from_pubkey == cosmos_address)

Signature: bT4/+FjxW435j9eNIJG+mJNF01/e0R3guqXKqHm+emQOEggBj7ygRO2c/iHbXuNilekG4HAlblfL+pATKdgHjQ==
Verification: True
Address Generated: cosmos1uspq8jesuu0f6uh6dez2lktnhuxrplzasuywag
Address Matched: False


In [20]:
cosmos_client = CosmosVerifier(bech_prefix="cosmos")

verification = cosmos_client.verify_signature(pubkey_b64, message_amino, sig_b64)
address_from_pubkey = cosmos_client.generate_address(pubkey_b64)

print("Signature:", sig_b64)
print("Verification:", verification)
print("Address Generated:", address_from_pubkey)
print("Address Matched:", address_from_pubkey == cosmos_address)

Signature: bT4/+FjxW435j9eNIJG+mJNF01/e0R3guqXKqHm+emQOEggBj7ygRO2c/iHbXuNilekG4HAlblfL+pATKdgHjQ==
Verification: True
Address Generated: cosmos1uspq8jesuu0f6uh6dez2lktnhuxrplzasuywag
Address Matched: False


In [21]:
ethereum_client = EthereumVerifier()

signature_from_metamask = "90ec75b7ec3fa034423c07d507242d1a719a4c97ea067ce8a093f001811c18396855d888ae3108eb0e23d3f186674463965c4a01d352db5492ecdfa9233980da1c"
sig_b64 = sign_ethereum(ethereum_private_key, message_amino).replace("0x", "")

verification = ethereum_client.verify_signature(ethereum_public_key, message_amino, sig_b64)
address_from_pubkey = ethereum_client.generate_address(ethereum_public_key)

print("Signature:", sig_b64)
print("Verification:", verification)
print("Address Generated:", address_from_pubkey)
print("Address Matched:", address_from_pubkey == ethereum_address)

ImportError: None of these hashing backends are installed: ['pycryptodome', 'pysha3'].
Install with `python -m pip install "eth-hash[pycryptodome]"`.

### Verifying from Keplr

In [ ]:
signature_from_keplr = "MEQCIBQR8ZIeFP2+/Xag924ASRNDRmrPf5QMAn8rd9dH5uFHAiARJBZGUOedjgvdympaoW1UC2dZ5mlngyH3uYRUoiQXIQ=="

sig_b64 = base64.b64decode(signature_from_keplr).hex()
verification = cosmos_client.verify_signature(cosmos_public_key, message_amino, sig_b64)
address_from_pubkey = cosmos_client.generate_address(cosmos_public_key)

print("Signature:", sig_b64)
print("Verification:", verification)